In [40]:
# !pip install paho-mqtt
# !pip install dnslib
# !pip install dnspython
# !python -m pip install --upgrade pip
!pip3 install pysocks==1.7.0

  Attempting uninstall: pysocks
    Found existing installation: PySocks 1.7.1
    Uninstalling PySocks-1.7.1:
      Successfully uninstalled PySocks-1.7.1


# Publisher

In [1]:
from http.server import SimpleHTTPRequestHandler
from socketserver import TCPServer
import json
import paho.mqtt.client as mqtt
import time
import os


In [5]:
import socks
import socket
client = mqtt.Client()    
client.username_pw_set('username', 'password')
# client.proxy_set(proxy_type=socks.PROXY_TYPE_HTTP, proxy_addr="192.168.10.13", proxy_port=8222)
socks.setdefaultproxy(socks.HTTP, '192.168.10.13', 8222)#"socks.proxy.host", 1080)
socket.socket = socks.socksocket
# connect
client.connect('abc.com', port=8080, keepalive=60, bind_address="")
client.loop_forever()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_17248\521666475.py:3: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


ProxyConnectionError: Error connecting to HTTP proxy 192.168.10.13:8222: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
broker_address = "broker.hivemq.com" # Free Available MQTT Broker
broker_port = 1883 # MQTT Default Port Number

client = mqtt.Client()
client.connect(broker_address, broker_port, 60) # 60s = Keep Alive interval is the duration to wait for ACK before retransmission. 

class MyHandler(SimpleHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()

            if pre_state[f'btn1']:
                button = (f"""
                        <div class="form-check form-switchmx-auto">
                          <input id="btn1" onclick="buttonClicked('btn1')" class="form-check-input" checked type="checkbox" style="width: 9em; height: 3em;">
                        </div>
                        """)
            else:
                button = (f"""
                        <div class="form-check form-switchmx-auto">
                          <input id="btn1" onclick="buttonClicked('btn1')" class="form-check-input" type="checkbox" style="width: 9em; height:3em;">
                        </div>
                        """)

            html_content = """
            <!DOCTYPE html>
            <html lang="en">
            
            <head>
              <meta charset="UTF-8">
              <meta name="viewport" content="width=device-width, initial-scale=1.0">
              <title>Smart Plug</title>
              <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.1/dist/css/bootstrap.min.css" rel="stylesheet"
              integrity="sha384-4bw+/aepP/YC94hEpVNVgiZdgIC5+VKNBQNGCHeKRQN+PtmoHDEXuppvnDJzQIu9" crossorigin="anonymous">
            </head>
            <body>

            <div class="container text-center">
              <br/>
              <h2><strong>Smart Plug</strong></h2>
              <div class="text-center">
                <br/>
                <h5>Toggle the button to control the switch:</h5>
                <br/><br/>
                <div>
                  """ + button + """
                </div>
              </div>
            </div>

            <script>
              function buttonClicked(buttonId) {
                var checkbox = document.getElementById(buttonId);
                var xhr = new XMLHttpRequest();
                xhr.open("GET", "/" + buttonId + "," + checkbox.checked,true);
                xhr.send();
              }
            </script>

            <script
              src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.1/dist/js/bootstrap.bundle.min.js"
              integrity="sha384-HwwvtgBNo3bZJJLYd8oVXjrBZt8cqVSpeBNS5n7C8IVInixGAoxmnlMuBnhbgrkm" crossorigin="anonymous">
            </script>

            </body>
            </html>
            """
            self.wfile.write(html_content.encode())
            
        else:
            button_id = self.path[1:] # Remove the leading '/'
            self.handle_button_click(button_id)

    def handle_button_click(self, btn_id):
        if btn_id[:3] == 'btn':
            btn, state = btn_id.split(',')
            s = 0
            pre_state[btn] = False
            if state == 'true':
                pre_state[btn] = True
                s = 1
            with open('states.json', 'w') as file:
                file.write(json.dumps(pre_state))

            topic = "CN_CEP/SmartPlug"
            message = f"{btn}{s}"
            
            print(s, topic, message)
            client.publish(topic, message)
            print(f"Button {btn} state: {state}")
            
if __name__ == "__main__":
    PORT = 8222
    
    if not os.path.isfile('states.json'): # Make json file to save pre_state of switch, if not exists
        pre_state = {"btn1": False} # Switch off by default
        with open('states.json', 'w') as file:
            file.write(json.dumps(pre_state))
    else: # Loads pre_state from existing json file
        with open('states.json') as file: 
            pre_state = json.loads(file.read())
            
    with TCPServer(("192.168.10.13", PORT), MyHandler) as httpd:
        print(f"Serving on port {PORT}")
        httpd.serve_forever()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4000\2798060928.py:4: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Serving on port 8222


192.168.10.13 - - [27/Dec/2024 06:20:19] "GET / HTTP/1.1" 200 -


1 CN_CEP/SmartPlug btn11
Button btn1 state: true
1 CN_CEP/SmartPlug btn11
Button btn1 state: true
0 CN_CEP/SmartPlug btn10
Button btn1 state: false


192.168.10.13 - - [27/Dec/2024 06:20:42] code 501, message Unsupported method ('CONNECT')
192.168.10.13 - - [27/Dec/2024 06:20:42] "CONNECT abc.com:8080 HTTP/1.1" 501 -
192.168.10.13 - - [27/Dec/2024 06:20:47] code 501, message Unsupported method ('CONNECT')
192.168.10.13 - - [27/Dec/2024 06:20:47] "CONNECT abc.com:8080 HTTP/1.1" 501 -
192.168.10.13 - - [27/Dec/2024 06:20:48] code 501, message Unsupported method ('CONNECT')
192.168.10.13 - - [27/Dec/2024 06:20:48] "CONNECT broker.hivemq.com:1883 HTTP/1.1" 501 -
192.168.10.13 - - [27/Dec/2024 06:21:33] code 501, message Unsupported method ('CONNECT')
192.168.10.13 - - [27/Dec/2024 06:21:33] "CONNECT abc.com:8081 HTTP/1.1" 501 -
192.168.10.13 - - [27/Dec/2024 06:21:45] code 501, message Unsupported method ('CONNECT')
192.168.10.13 - - [27/Dec/2024 06:21:45] "CONNECT abc.com:1883 HTTP/1.1" 501 -
192.168.10.13 - - [27/Dec/2024 06:22:10] code 501, message Unsupported method ('CONNECT')
192.168.10.13 - - [27/Dec/2024 06:22:10] "CONNECT abc.c

1 CN_CEP/SmartPlug btn11
Button btn1 state: true
0 CN_CEP/SmartPlug btn10
Button btn1 state: false


192.168.10.13 - - [27/Dec/2024 06:27:26] code 501, message Unsupported method ('CONNECT')
192.168.10.13 - - [27/Dec/2024 06:27:26] "CONNECT cep.cn:80 HTTP/1.1" 501 -
192.168.10.13 - - [27/Dec/2024 06:30:19] code 501, message Unsupported method ('CONNECT')
192.168.10.13 - - [27/Dec/2024 06:30:19] "CONNECT cep.cn:80 HTTP/1.1" 501 -
192.168.10.13 - - [27/Dec/2024 06:30:37] code 501, message Unsupported method ('CONNECT')
192.168.10.13 - - [27/Dec/2024 06:30:37] "CONNECT cep.cn:80 HTTP/1.1" 501 -
192.168.10.13 - - [27/Dec/2024 06:32:38] "GET / HTTP/1.1" 200 -
192.168.10.13 - - [27/Dec/2024 06:32:39] "GET / HTTP/1.1" 200 -


1 CN_CEP/SmartPlug btn11
Button btn1 state: true
0 CN_CEP/SmartPlug btn10
Button btn1 state: false


192.168.10.13 - - [27/Dec/2024 06:34:45] code 501, message Unsupported method ('CONNECT')
192.168.10.13 - - [27/Dec/2024 06:34:45] "CONNECT cep_smartplug.cn:80 HTTP/1.1" 501 -


# DNSServer

In [23]:
from http.server import SimpleHTTPRequestHandler
from dnslib.server import DNSServer
import json
import paho.mqtt.client as mqtt
import time
import os

from dns.resolver import Resolver


In [24]:
broker_address = "broker.hivemq.com" # Free Available MQTT Broker
broker_port = 1883 # MQTT Default Port Number

client = mqtt.Client()
client.connect(broker_address, broker_port, 60) # 60s = Keep Alive interval is the duration to wait for ACK before retransmission. 

class MyHandler(SimpleHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()

            if pre_state[f'btn1']:
                button = (f"""
                        <div class="form-check form-switchmx-auto">
                          <input id="btn1" onclick="buttonClicked('btn1')" class="form-check-input" checked type="checkbox" style="width: 9em; height: 3em;">
                        </div>
                        """)
            else:
                button = (f"""
                        <div class="form-check form-switchmx-auto">
                          <input id="btn1" onclick="buttonClicked('btn1')" class="form-check-input" type="checkbox" style="width: 9em; height:3em;">
                        </div>
                        """)

            html_content = """
            <!DOCTYPE html>
            <html lang="en">
            
            <head>
              <meta charset="UTF-8">
              <meta name="viewport" content="width=device-width, initial-scale=1.0">
              <title>Smart Plug</title>
              <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.1/dist/css/bootstrap.min.css" rel="stylesheet"
              integrity="sha384-4bw+/aepP/YC94hEpVNVgiZdgIC5+VKNBQNGCHeKRQN+PtmoHDEXuppvnDJzQIu9" crossorigin="anonymous">
            </head>
            <body>

            <div class="container text-center">
              <br/>
              <h2><strong>Smart Plug</strong></h2>
              <div class="text-center">
                <br/>
                <h5>Toggle the button to control the switch:</h5>
                <br/><br/>
                <div>
                  """ + button + """
                </div>
              </div>
            </div>

            <script>
              function buttonClicked(buttonId) {
                var checkbox = document.getElementById(buttonId);
                var xhr = new XMLHttpRequest();
                xhr.open("GET", "/" + buttonId + "," + checkbox.checked,true);
                xhr.send();
              }
            </script>

            <script
              src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.1/dist/js/bootstrap.bundle.min.js"
              integrity="sha384-HwwvtgBNo3bZJJLYd8oVXjrBZt8cqVSpeBNS5n7C8IVInixGAoxmnlMuBnhbgrkm" crossorigin="anonymous">
            </script>

            </body>
            </html>
            """
            self.wfile.write(html_content.encode())
            
        else:
            button_id = self.path[1:] # Remove the leading '/'
            self.handle_button_click(button_id)

    def handle_button_click(self, btn_id):
        if btn_id[:3] == 'btn':
            btn, state = btn_id.split(',')
            s = 0
            pre_state[btn] = False
            if state == 'true':
                pre_state[btn] = True
                s = 1
            with open('states.json', 'w') as file:
                file.write(json.dumps(pre_state))

            topic = "CN/CEP"
            message = f"{btn}{s}"
            
            client.publish(topic, message)
            print(f"Button {btn} state: {state}")
            
if __name__ == "__main__":
    PORT = 8011
    
    if not os.path.isfile('states.json'): # Make json file to save pre_state of switch, if not exists
        pre_state = {"btn1": False} # Switch off by default
        with open('states.json', 'w') as file:
            file.write(json.dumps(pre_state))
    else: # Loads pre_state from existing json file
        with open('states.json') as file: 
            pre_state = json.loads(file.read())

    resolver = Resolver()
    resolver.nameservers = ['8.8.8.8']
    
    server = DNSServer(resolver, address="localhost", port=PORT)
#     MyHandler
    server.start_thread()
    print( server.isAlive() )
#     while server.is_running:
#         # this just keeps the thing alive...
#         time.sleep(5)
#     server.stop()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_35400\3530774292.py:4: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


True


In [12]:
from dnslib.dns import DNSRecord

q = DNSRecord.question("smartplug.cncep")
a = q.reply()
a.add_answer(*RR.fromZone("smartplug.cncep A 192.168.10.13"))

print(a)


;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 30164
;; flags: qr aa rd ra; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 0
;; QUESTION SECTION:
;smartplug.cncep.               IN      A
;; ANSWER SECTION:
smartplug.cncep.        0       IN      A       192.168.10.13


In [1]:
from dnslib.server import DNSServer, DNSLogger
from dnslib.dns import RR

class TestResolver:
    def resolve(self, request, handler):
        reply = request.reply()
        reply.add_answer(*RR.fromZone("abc.def. 60 A 1.2.3.4"))
        return reply
    
resolver = TestResolver()
logger = DNSLogger(prefix=False)
server = DNSServer(resolver, port=8222, address="192.168.10.6", logger=logger, tcp=True)
server.start_thread()
a = q.send("192.168.10.6", 8222, tcp=True)

print(DNSRecord.parse(a))

server.stop()

OSError: [WinError 10049] The requested address is not valid in its context

# Base

In [ ]:
from http.server import SimpleHTTPRequestHandler
from socketserver import TCPServer
import json
import paho.mqtt.client as mqtt
import time
import os

broker_address = "broker.hivemq.com"
broker_port = 1883
N = 1 # Number of buttons

client = mqtt.Client()

client.connect(broker_address, broker_port, 60)
class MyHandler(SimpleHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            btns = []
            for i in range(1, N+1):
                if pre_state[f'btn{i}']:
                    btns.append(f"""<div class="form-check form-switch
                    mx-auto"><input id="btn{i}" onclick="buttonClicked('btn{i}')"
                    class="form-check-input" checked type="checkbox" style="width: 9em;
                    height: 3em;"></div>""")
                else:
                    btns.append(f"""<div class="form-check form-switch
                    mx-auto"><input id="btn{i}" onclick="buttonClicked('btn{i}')"
                    class="form-check-input" type="checkbox" style="width: 9em; height:
                    3em;"></div>""")
            html_content = """9
            <!DOCTYPE html>
            <html lang="en">
            <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width,
            initial-scale=1.0">
            <title>Smart Plug</title>
            <link
            href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.1/dist/css/bootstrap.mi
            n.css" rel="stylesheet"
            integrity="sha384-4bw+/aepP/YC94hEpVNVgiZdgIC5+VKNBQNGCHeKRQN+PtmoHDEXup
            pvnDJzQIu9" crossorigin="anonymous">
            </head>
            <body>
            <nav class="navbar navbar-expand-lg bg-dark border-bottom
            border-body text-center" data-bs-theme="dark">
            <div class="container text-center">
            <a>.</a>
            </div>
            </nav>
            <div class="container text-center">
            <br/>
            <h2><strong> Welcome to Fahad's Smart Plug</strong></h2>
            <div class="text-center">
            <br/>
            <h5>Toggle a button to control the corresponding
            appliance:</h5>
            <br/>
            <br/>
            <div class="row gap-3 justify-content-center">
            """ + "\n".join(btns) + """
            </div>
            </div>
            </div>
            <script>
            function buttonClicked(buttonId) {
            var checkbox = document.getElementById(buttonId);
            var xhr = new XMLHttpRequest();10
            xhr.open("GET", "/" + buttonId + "," + checkbox.checked,
            true);
            xhr.send();
            }
            </script>
            <script
            src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.1/dist/js/bootstrap.bund
            le.min.js"
            integrity="sha384-HwwvtgBNo3bZJJLYd8oVXjrBZt8cqVSpeBNS5n7C8IVInixGAoxmnl
            MuBnhbgrkm" crossorigin="anonymous"></script>
            </body>
            </html>
            """
            self.wfile.write(html_content.encode())
        else:
            button_id = self.path[1:] # Remove the leading '/'
            print(button_id)
            self.handle_button_click(button_id)
    def handle_button_click(self, btn_id):
        if btn_id[:3] == 'btn':
            btn, state = btn_id.split(',')
            s = 0
            pre_state[btn] = False
            if state == 'true':
                pre_state[btn] = True
                s = 1
            with open('states.json', 'w') as file:
                file.write(json.dumps(pre_state))
            topic = "CN/CEP"
            message = f"{btn[-1]}{s}"
            client.publish(topic, message)
            print(f"Button {btn[-1]} state: {state}")
            
if __name__ == "__main__":
    PORT = 8011
    if not os.path.isfile('states.json'):
        pre_state = {f"btn{i+1}": False for i in range(N)}
        with open('states.json', 'w') as file:
            file.write(json.dumps(pre_state))
    else:
        with open('states.json') as file:
            pre_state = json.loads(file.read())
    with TCPServer(("localhost", PORT), MyHandler) as httpd:
        print(f"Serving on port {PORT}")
        httpd.serve_forever()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33112\883578377.py:12: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Serving on port 8011


127.0.0.1 - - [25/Dec/2024 09:30:44] "GET / HTTP/1.1" 200 -


favicon.ico
favicon.ico


127.0.0.1 - - [25/Dec/2024 09:30:58] "GET / HTTP/1.1" 200 -


favicon.ico
favicon.ico


127.0.0.1 - - [25/Dec/2024 09:31:01] "GET / HTTP/1.1" 200 -


favicon.ico
favicon.ico
btn1,false
Button 1 state: false
btn1,true
Button 1 state: true
btn1,true
Button 1 state: true
